# Clustering performance evaluation
這次要介紹在分群 clustering 任務上的評價指標。

## rand_score
第一個要介紹的就是 rand_score ，這個方法跟給標記的順序無關。

In [2]:
from sklearn.metrics import rand_score

labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

display(rand_score(labels_true, labels_pred))

labels_pred = [1, 1, 0, 0, 3, 3]

display(rand_score(labels_true, labels_pred))


0.6666666666666666

0.6666666666666666

## adjusted_rand_score

In [3]:
from sklearn.metrics import adjusted_rand_score

labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

display(adjusted_rand_score(labels_true, labels_pred))

labels_pred = [1, 1, 0, 0, 3, 3]

display(adjusted_rand_score(labels_true, labels_pred))

0.24242424242424243

0.24242424242424243